In [1]:
from openai import OpenAI
import os
import time
import pandas as pd
from tqdm import tqdm
from utils.preprocessing import main_preprocessing_activity_recommendation

In [2]:
api_key='sk-proj-QhecSyhgkItKRIervRJ9T3BlbkFJnVhbp8sOqtRavkGwqlxx'
os.environ['OPENAI_API_KEY'] =api_key

In [3]:
client = OpenAI()

In [4]:
llm_name = 'gpt-4'#'Phi-3-14b'#'falcon2-11b' #'claude2-13b' # 'mistral-7b-instruct'#'llama3-7b' # 'mixtral-8x7b-instruct'
# RPA
bpm_task='rpa'
prompt_rpa_persona = {'prompt_pattern':'persona',
                 'bpm_task':bpm_task,
                 'model_name':llm_name, 
                 'user_prompt':'The process activity is: ',
                 'system_prompt':'You are a specialist in process mining and an expert in identifying robotic process automation potential. A user provides you with a process activity. Classify this activity as 0 if it is a manual task (done by a user without any information system), 1 if it is a user task (done by a user on an information system), 2 if it is an automation task ( done on an information system without any user involved). Only response with the number, without an explanation.'}
prompt_rpa_few_shot = {'prompt_pattern':'few_shot',
                 'bpm_task':bpm_task,
                 'model_name':llm_name, 
                 'user_prompt':'The process activity is: ',
                 'system_prompt':'A user provides you with a process activity. Classify this activity as 0 if it is a manual task (done by a user without any information system), 1 if it is a user task (done by a user on an information system), 2 if it is an automation task ( done on an information system without any user involved). For example: "Filing physical documents in a cabinet" - 0 (manual task), "Entering customer data into a CRM system" - 1 (user task), "Generating monthly financial reports automatically" - 2 (automation task). Only response with the number, without an explanation.'}
prompt_rpa_step_by_step = {'prompt_pattern':'step_by_step',
                 'bpm_task':bpm_task,
                 'model_name':llm_name, 
                 'user_prompt':'The process activity is: ',
                 'system_prompt':'A user provides you with a process activity. Classify this activity as 0 if it is a manual task (done by a user without any information system), 1 if it is a user task (done by a user on an information system), 2 if it is an automation task ( done on an information system without any user involved). Let\'s think step by step to solve the problem. Only response with the number, without an explanation.'}
# CONSTRAINTS
bpm_task='constraints'
prompt_declare_persona = {'prompt_pattern':'persona',
                 'bpm_task':bpm_task,
                 'model_name':llm_name, 
                 'user_prompt':'The process activity is: ',
                 'system_prompt':'You are a specialist in process mining and an expert in identifying declare constraints. A user provides you with a process activity. Classify this activity as "precedence" (one activity must happen before another), "response" (one activity must happen after another), "succession" (one activity must happen before and be followed by another), "end" (an activity marks the end of the process), or "init" (an activity marks the beginning of the process). Only provide the constraint type, without an explanation.'}
prompt_declare_few_shot = {'prompt_pattern':'few_shot',
                 'bpm_task':bpm_task,
                 'model_name':llm_name, 
                 'user_prompt':'The process activity is: ',
                 'system_prompt':'A user provides you with a process activity. Classify this activity as "precedence" (one activity must happen before another), "response" (one activity must happen after another), "succession" (one activity must happen before and be followed by another), "end" (an activity marks the end of the process), or "init" (an activity marks the beginning of the process). For example: "Whenever the sales department receives an order, a new process instance is created." - "init", "First, the supervisor approves within 7 days, this will result in the deposit of the reimbursement directly to the employee\'s bank account." - "response". Only provide the constraint type, without an explanation.'}
prompt_declare_step_by_step = {'prompt_pattern':'step_by_step',
                 'bpm_task':bpm_task,
                 'model_name':llm_name, 
                 'user_prompt':'The process activity is: ',
                 'system_prompt':'A user provides you with a process activity. Classify this activity as "precedence" (one activity must happen before another), "response" (one activity must happen after another), "succession" (one activity must happen before and be followed by another), "end" (an activity marks the end of the process), or "init" (an activity marks the beginning of the process). Let\'s think step by step to solve the problem. Only provide the constraint type, without an explanation.'}
# ACTIVITY RECOMMENDATION
bpm_task='activity_recommendation'
prompt_activity_recommendation_persona = {'prompt_pattern':'persona',
                 'bpm_task':bpm_task,
                 'model_name':llm_name, 
                 'user_prompt':'The sequence of process activities is: ',
                 'system_prompt':'You are a specialist in process mining and an expert in recommending the next best activity. A user provides you with a sequence of three process activities separated by commas. Recommend an activity that fits best as the next activity in the sequence. Only provide the activity, without an explanation.'}
prompt_activity_recommendation_few_shot = {'prompt_pattern':'few_shot',
                 'bpm_task':bpm_task,
                 'model_name':llm_name, 
                 'user_prompt':'The sequence of process activities is: ',
                 'system_prompt':'A user provides you with a sequence of three process activities separated by commas. Recommend an activity that fits best as the next activity in the sequence. For example: "Receive order, Verify order, Process payment" - "Ship order", "Log in, Navigate to dashboard, Generate report" - "Log out". Only provide the activity, without an explanation.'}
prompt_activity_recommendation_step_by_step = {'prompt_pattern':'step_by_step',
                 'bpm_task':bpm_task,
                 'model_name':llm_name, 
                 'user_prompt':'The sequence of process activities is: ',
                 'system_prompt':'A user provides you with a sequence of three process activities separated by commas. Recommend an activity that fits best as the next activity in the sequence. Let\'s think step by step to solve the problem. Only provide the activity, without an explanation.'}
# PROCESS QUESTION ANSWERING
bpm_task = 'process_qa'
prompt_process_qa_persona = {'prompt_pattern': 'persona',
                                   'bpm_task': bpm_task,
                                   'model_name': llm_name,
                                   'user_prompt': 'The process description is: ',
                                   'system_prompt': 'You are a specialist in process mining and an expert in analyzing process descriptions. A user provides you with a process description and a related question. Answer the question based on the process description provided.'}
prompt_process_qa_few_shot = {'prompt_pattern': 'few_shot',
                                    'bpm_task': bpm_task,
                                    'model_name': llm_name,
                                    'user_prompt': 'The process description is: ',
                                    'system_prompt': 'A user provides you with a process description and a related question. Answer the question based on the process description provided. For example: The process description is: "The process involves receiving an order, verifying the order, processing the payment, and shipping the order." Question: "What is the first step in the process?" Answer: "Receiving an order". Only provide the answer.'}
prompt_process_qa_step_by_step = {'prompt_pattern': 'step_by_step',
                                        'bpm_task': bpm_task,
                                        'model_name': llm_name,
                                        'user_prompt': 'The process description is: ',
                                        'system_prompt': 'A user provides you with a process description and a related question. Answer the question based on the process description provided. Let\'s think step by step to solve the problem. Only provide the answer.'}



In [5]:
#load dataset 
# RPA
df_rpa = pd.read_excel('../data/grohs2023_llm-can-accomplish-bpm-tasks/rpa_task_classification.xlsx', sheet_name='Sheet1')
df_rpa = {'user_message': df_rpa['task'], 'true_label': df_rpa['manual_labels'], 'note':len(df_rpa['manual_labels'])* ['']}

# CONSTRAINTS
df_constraints = pd.read_excel('../data/grohs2023_llm-can-accomplish-bpm-tasks/text_to_ltl.xlsx', sheet_name='Sheet1')
df_constraints = {'user_message': df_constraints['task'], 'true_label': df_constraints['manual_labels'], 'note':len(df_constraints['manual_labels'])* ['']}

# ACTIVITY RECOMMENDATION
folder_path = '../Kiran SAP/constraintCheckingUsingLL/constraintCheckingUsingLLM/constraints-transformer/data/sap_sam_2022/filtered/logs/'
path_to_bpm_recommendation_dataset='data/bpm_activity_recommendataion.xlsx'
if not os.path.isfile(path_to_bpm_recommendation_dataset):
    df_recommendation = main_preprocessing_activity_recommendation(folder_path, num_files=300, num_processes=300, num_sequences_per_trace=1, sequence_length=4, save = True)
else:
    df_recommendation = pd.read_excel(path_to_bpm_recommendation_dataset)
df_recommendation = {'user_message': df_recommendation['task'], 'true_label': df_recommendation['manual_labels'], 'note':len(df_recommendation['manual_labels'])* ['']}

# PROCESS QA
df_qa = pd.read_excel('data/bpm_qa.xlsx')
df_qa['task'] = df_qa['process_description'] + " Question: "+ df_qa['question']
df_qa = {'user_message': df_qa['task'], 'true_label': df_qa['answer'], 'note':df_qa['level']}

In [6]:
prompts = [(prompt_rpa_persona, df_rpa),(prompt_rpa_few_shot, df_rpa),(prompt_rpa_step_by_step, df_rpa), # RPA 
           (prompt_declare_persona, df_constraints),(prompt_declare_few_shot, df_constraints),(prompt_declare_step_by_step, df_constraints), # CONSTRAINTS
           (prompt_activity_recommendation_persona, df_recommendation),(prompt_activity_recommendation_few_shot, df_recommendation),(prompt_activity_recommendation_step_by_step, df_recommendation), # ACTIVITY RECOMMENDATION
           (prompt_process_qa_persona, df_qa),(prompt_process_qa_few_shot, df_qa),(prompt_process_qa_step_by_step, df_qa), # PROCESS QA
           ]

In [7]:
inferance_time=[]
content=[]
true_value=[]
completion_tokens=[]
prompt_tokens=[]
total_tokens=[]
prompt_pattern=[]
bpm_task=[]
model_name=[]
user_input = []
note=[]
for prompt, df in prompts:
    for i, v in tqdm(enumerate(list(df['user_message']))):
        inferance_start = time.time()
        completion = client.chat.completions.create(
        model="gpt-4", 
        messages=[
            {"role": "system", "content": prompt['system_prompt']},
            {"role": "user", "content": prompt['user_prompt']+ v}
        ],
        temperature=0,
        )
        inferance_end = time.time()
        user_input.append(v)
        prompt_pattern.append(prompt['prompt_pattern'])
        bpm_task.append(prompt['bpm_task'])
        model_name.append(prompt['model_name'])
        note.append(df['note'][i])
        inferance_time.append(inferance_end - inferance_start)
        content.append(completion.to_dict()['choices'][0]['message']['content'])
        true_value.append(df['true_label'][i])
        completion_tokens.append(completion.to_dict()['usage']['completion_tokens'])
        prompt_tokens.append(completion.to_dict()['usage']['prompt_tokens'])
        total_tokens.append(completion.to_dict()['usage']['total_tokens'])


424it [05:42,  1.24it/s]
424it [05:24,  1.31it/s]
424it [05:18,  1.33it/s]
104it [01:20,  1.29it/s]
104it [01:26,  1.20it/s]
104it [01:25,  1.21it/s]
288it [04:38,  1.03it/s]
288it [04:29,  1.07it/s]
288it [04:43,  1.02it/s]
60it [05:46,  5.78s/it]
60it [02:46,  2.78s/it]
60it [03:38,  3.64s/it]


In [8]:
result = {'prompt_pattern':prompt_pattern,
          'user_input':user_input,
'bpm_task':bpm_task,
'model_name':model_name,
'inferance_time':inferance_time,
'content':content,
'true_value':true_value,
'completion_tokens':completion_tokens,
'prompt_tokens':prompt_tokens,
'total_tokens':total_tokens, 
'note':note}

In [9]:
df = pd.DataFrame(result)

In [10]:
df

,prompt_pattern,user_input,bpm_task,model_name,inferance_time,content,true_value,completion_tokens,prompt_tokens,total_tokens,note
0,persona,manufacture customized bicycles,rpa,gpt-4,0.958288,0,0,1,115,116,
1,persona,receive an order,rpa,gpt-4,0.744916,1,1,1,115,116,
2,persona,create a new process instance,rpa,gpt-4,0.886625,1,1,1,117,118,
3,persona,reject the order,rpa,gpt-4,0.870642,1,1,1,115,116,
4,persona,accept the order,rpa,gpt-4,1.081032,1,1,1,115,116,
...,...,...,...,...,...,...,...,...,...,...,...
2623,step_by_step,A guest enters the restaurant when feeling hun...,process_qa,gpt-4,14.316707,The process flow starts when a guest feels hun...,The guest enters the restaurant when feeling h...,184,296,480,complex
2624,step_by_step,A guest enters the restaurant when feeling hun...,process_qa,gpt-4,14.310467,Potential points of failure in this process co...,Potential points of failure include the guest ...,205,299,504,complex
2625,step_by_step,A guest enters the restaurant when feeling hun...,process_qa,gpt-4,4.091510,The POS system plays a critical role in this p...,The POS system records the order and processes...,44,292,336,complex
2626,step_by_step,A guest enters the restaurant when feeling hun...,process_qa,gpt-4,8.968426,The roles of the employee and the chef are int...,The employee and chef must communicate effecti...,124,295,419,complex


In [11]:
df[df['bpm_task']=='constraints']

,prompt_pattern,user_input,bpm_task,model_name,inferance_time,content,true_value,completion_tokens,prompt_tokens,total_tokens,note
1272,persona,"A claim should be created, before it can be ap...",constraints,gpt-4,1.096314,precedence,"precedence ,, ,",2,133,135,
1273,persona,"A claim must be created, before it is approved.",constraints,gpt-4,1.091055,precedence,"precedence ,, ,",2,132,134,
1274,persona,"When a claim is created, it may be approved.",constraints,gpt-4,0.718475,Response,"precedence ,, ,",1,132,133,
1275,persona,"If a claim is created, it can be approved.",constraints,gpt-4,0.664642,Response,"precedence ,, ,",1,132,133,
1276,persona,"They have to create the claim, before they can...",constraints,gpt-4,0.603865,precedence,"precedence ,, ,",2,134,136,
...,...,...,...,...,...,...,...,...,...,...,...
1579,step_by_step,"Once the expense report is received, a check i...",constraints,gpt-4,0.927558,precedence,"response ,, ,",2,137,139,
1580,step_by_step,If this check does not take place within 28 da...,constraints,gpt-4,0.703959,response,"response ,, ,",1,142,143,
1581,step_by_step,"First, the supervisor approves within 7 days, ...",constraints,gpt-4,0.794046,precedence,"response ,, ,",2,144,146,
1582,step_by_step,"Second, the supervisor rejects within 7 days, ...",constraints,gpt-4,0.734227,response,"response ,, ,",1,144,145,


In [12]:
df['bpm_task']

0              rpa
1              rpa
2              rpa
3              rpa
4              rpa
           ...    
2623    process_qa
2624    process_qa
2625    process_qa
2626    process_qa
2627    process_qa
Name: bpm_task, Length: 2628, dtype: object

In [13]:
df.to_excel(f'result_{llm_name}.xlsx', index=False)